In [1]:
import torch
import torchvision
import construct
import prep
import pandas as pd

In [2]:
image_dir = 'data/10x'
labels_file = 'data/me.csv'

DATA = prep.prep_data(pd.read_csv(labels_file), image_dir)

transforms = torchvision.transforms.Compose([
                            torchvision.transforms.ToPILImage(),
                            torchvision.transforms.RandomRotation((-180,180)),
                            torchvision.transforms.CenterCrop((325)),
                            torchvision.transforms.ToTensor()
                                      ])

/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = False
/Users/Will/GradSchool/CHEME/Project/FmPM/fmpm/will/prep.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  identification[i] = True


In [80]:
class myModel(torch.nn.Module):
    def __init__(self):
        """
        Initializes CNN. Here we just define layer shapes that we call in the forward func
        """
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
                
        #Convultion layer 2. See above
        self.conv2 = torch.nn.Conv2d(in_channels = 6, 
                               out_channels = 12, 
                               kernel_size = 5)
        
        self.fc_1 = torch.nn.Linear(39 * 39 * 12, 256)
        self.fc_2 = torch.nn.Linear(256, 2)
        self.drop = torch.nn.Dropout(p=.2)
        self.batch1 = torch.nn.BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch2 = torch.nn.BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            
    def forward(self, x):
        """
        Function that performs all the neural network forward calculation i.e.
        takes image data from the input of the neural network to the output
        """
        
        x = self.conv1(x)
        x = self.batch1(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 2)
        x = torch.nn.functional.leaky_relu(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = torch.nn.functional.max_pool2d(x, kernel_size = 4)
        x = torch.nn.functional.leaky_relu(x)
        x = x.view(x.shape[0], -1)  
        x = self.fc_1(x) 
        x = torch.nn.functional.leaky_relu(x)
        x = self.drop(x)
        x = self.fc_2(x) 
        return x

model = myModel()

In [81]:
#Declaring iterator. The thing that will loop through our dataset.

data = prep.tenX_dataset(DATA, 'data/10x', transform=transforms)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test, train = torch.utils.data.random_split(data, [int(.2*len(data)), len(data)-int(.2*len(data))])
BATCH_SIZE = 32
epochs = 150

In [82]:
cnn = construct.train(epochs, BATCH_SIZE, train, criterion, 'Adam', model)

EPOCH: 0, acc: 0.6979166641831398, loss: 5.750095374882221
tensor([[ 2.3094e+01, -1.0397e+01],
        [ 5.9005e+00, -3.4715e+00],
        [ 1.7638e+01, -8.3825e-03],
        [ 1.0006e+01, -1.0271e+01],
        [ 1.6215e+01, -1.1620e+00],
        [ 1.0440e+01, -5.0848e+00],
        [ 1.3878e+01, -7.9502e+00],
        [ 1.0961e+01,  1.3480e+00],
        [ 3.7526e+00, -1.9090e+00],
        [ 5.3434e-01,  1.1362e+00],
        [ 9.4027e+00, -2.2097e+00],
        [ 8.3002e+00, -3.7091e+00],
        [-6.1417e-01,  7.8141e+00],
        [ 1.0663e+01, -5.5090e+00],
        [ 5.8044e+00,  8.1543e+00],
        [ 7.8463e+00, -6.4460e+00],
        [ 2.3842e+01,  7.9774e+00],
        [ 4.3242e+00,  9.9970e-01],
        [ 6.4993e+00,  3.1583e+00],
        [ 1.3074e+00,  1.7914e+00],
        [ 9.7586e+00, -6.9590e+00],
        [ 7.0269e+00,  1.9315e+00],
        [ 4.7738e+00, -3.5179e+00],
        [ 9.1450e+00,  1.0386e+01]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
        [1, 0],
        [1, 0],
   

EPOCH: 26, acc: 0.8919270858168602, loss: 0.25420425087213516
EPOCH: 27, acc: 0.890625, loss: 0.27722874842584133
EPOCH: 28, acc: 0.89453125, loss: 0.235028188675642
EPOCH: 29, acc: 0.8684895858168602, loss: 0.3140815831720829
EPOCH: 30, acc: 0.88671875, loss: 0.2691895607858896
tensor([[ 5.2089, -6.0855],
        [ 3.9888,  0.6532],
        [-0.8267, -0.3874],
        [ 1.4514, -2.1076],
        [ 6.0810, -1.8076],
        [ 3.0499,  1.1116],
        [ 9.7073,  0.0472],
        [ 1.3148,  0.6247],
        [ 7.4300,  4.2967],
        [-2.8205,  0.6595],
        [-4.6652,  0.0132],
        [ 2.9102,  0.4580],
        [10.0552,  3.1777],
        [ 0.4203,  2.2408],
        [ 7.0338,  1.7584],
        [-1.6919,  0.2374],
        [ 3.2987,  0.8316],
        [ 6.6143, -0.2492],
        [-0.8444,  0.8012],
        [ 1.8737,  1.9244],
        [-0.5020,  1.7270],
        [ 3.4086,  0.3148],
        [ 5.2279, -2.5712],
        [ 8.8421, -3.0950]], grad_fn=<AddmmBackward>)
tensor([[1, 0],
      

EPOCH: 56, acc: 0.8854166641831398, loss: 0.2445057537406683
EPOCH: 57, acc: 0.8723958358168602, loss: 0.29491529054939747
EPOCH: 58, acc: 0.9153645858168602, loss: 0.21981496457010508
EPOCH: 59, acc: 0.8489583358168602, loss: 0.3124406049028039
EPOCH: 60, acc: 0.8802083358168602, loss: 0.44761565513908863
tensor([[ -1.5855,   1.7606],
        [  0.3275,  -5.0572],
        [  0.4252,  -0.1436],
        [  1.7624,  -8.5119],
        [  1.3141,  -9.3683],
        [ -0.3159,  -0.5964],
        [  5.1738,  -4.9290],
        [  3.5901,  -2.3002],
        [  4.0436,  -7.0357],
        [  0.1203,   4.1378],
        [  3.6058,  -9.4327],
        [  2.7820,  -9.8014],
        [  0.8276,  -0.9351],
        [  4.6348,  -6.1569],
        [  7.4406, -10.3636],
        [  0.4614,  -1.6595],
        [  1.0992,   0.4527],
        [  1.9588,  -3.7208],
        [  2.6350,   0.0703],
        [ -1.6416,   0.0763],
        [  3.7801,  -5.6464],
        [  1.6698,  -7.0038],
        [  1.6819,  -1.4026],
  

EPOCH: 86, acc: 0.8984375, loss: 0.20878117345273495
EPOCH: 87, acc: 0.9127604141831398, loss: 0.2034709108993411
EPOCH: 88, acc: 0.94140625, loss: 0.17194968927651644
EPOCH: 89, acc: 0.9270833358168602, loss: 0.20423799939453602
EPOCH: 90, acc: 0.9244791641831398, loss: 0.1611128617078066
tensor([[ 2.8289e+00, -1.3545e+01],
        [-3.6062e-01, -1.6167e+00],
        [ 5.6080e-01, -3.0564e+00],
        [-2.8356e+00,  1.6340e+00],
        [ 3.2617e+00, -3.7232e+00],
        [ 4.6092e+00, -1.7331e+00],
        [ 4.9430e+00, -5.9724e+00],
        [ 9.4299e+00, -9.4467e+00],
        [ 2.5787e+00, -2.5274e-01],
        [ 1.7447e+00, -1.4553e+00],
        [ 2.1277e+00, -1.0721e+00],
        [ 4.2001e+00, -1.6316e+01],
        [-5.3627e+00,  1.4900e+00],
        [ 2.0211e+00, -3.3080e+00],
        [ 5.4244e+00, -1.3920e+00],
        [-3.7162e+00, -1.0542e-01],
        [ 1.1470e+01, -1.3506e+01],
        [ 2.4916e+00,  7.0527e-01],
        [ 5.6266e+00,  6.4756e-01],
        [ 1.9333e+00,  1.

EPOCH: 116, acc: 0.921875, loss: 0.18638732563704252
EPOCH: 117, acc: 0.91796875, loss: 0.16370293777436018
EPOCH: 118, acc: 0.91015625, loss: 0.22796664666384459
EPOCH: 119, acc: 0.9361979141831398, loss: 0.17105094343423843
EPOCH: 120, acc: 0.94921875, loss: 0.15107929287478328
tensor([[ 1.5404e+00, -1.3868e+00],
        [ 1.8840e+00,  2.2349e+00],
        [ 2.7874e+00, -6.1584e+00],
        [ 3.5649e-01, -3.8061e+00],
        [-1.0195e+00, -8.7470e-01],
        [ 6.6865e+00, -1.1480e+01],
        [ 1.8414e+00, -1.2928e+00],
        [ 6.6004e+00, -1.8005e+01],
        [ 7.4185e-02, -7.7571e+00],
        [ 1.9947e+00, -1.8082e+00],
        [ 1.7157e+00, -1.4855e+00],
        [-2.9445e+00,  9.4961e-01],
        [-9.6981e-02, -3.0247e-01],
        [ 1.7833e+00, -3.3149e+00],
        [ 8.5070e+00, -1.7834e+01],
        [ 2.9682e+00, -2.1662e+00],
        [-1.2907e+00, -1.9431e+00],
        [ 5.8333e-01, -6.0688e-01],
        [-3.9533e+00, -2.1367e+00],
        [ 5.9391e+00, -1.2037e+01],

EPOCH: 146, acc: 0.9296875, loss: 0.16384555120021105
EPOCH: 147, acc: 0.9296875, loss: 0.16171221900731325
EPOCH: 148, acc: 0.9244791641831398, loss: 0.1990489326417446
EPOCH: 149, acc: 0.91015625, loss: 0.22113266959786415
EPOCH: 150, acc: 0.9010416641831398, loss: 0.263285243883729
tensor([[  5.6262,  -3.8316],
        [  7.9056,  -9.8959],
        [ -0.1170,   1.8349],
        [  1.8415,  -1.6661],
        [  0.2560,  -0.5614],
        [  0.2014,   0.6122],
        [  1.4245,  -0.8340],
        [  5.0562,  -3.3810],
        [  2.2500,  -1.3283],
        [  2.7472,  -0.4444],
        [  8.1968, -15.6105],
        [  1.6785,  -1.7524],
        [  7.4304,  -5.5714],
        [  7.3737,  -8.9918],
        [  2.1155,  -1.4570],
        [  7.9337,  -2.5250],
        [  7.0220,  -4.7393],
        [ -0.3718,   5.2064],
        [  6.9710,  -6.7701],
        [  5.7829,  -2.2353],
        [  0.8168,  -0.7911],
        [  7.4372,  -4.9040],
        [ -1.2001,  -0.6336],
        [  6.8602,  -6.4

In [83]:
images, labels, predictions, weights, acc = construct.get_predictions(BATCH_SIZE, cnn, test)

In [84]:
(labels[:,1] == predictions).float().sum()/len(predictions)

tensor(0.8689)

In [85]:
(labels[:,1] == 0).float().sum()/len(predictions)

tensor(0.8689)

In [ ]:
def test_dataset_class():
    filename = '../tests/test_assets/test_labels_unclean.csv'
    image_dir = '../tests/test_assets/test_images'
    
    #Eventually just read in already cleaned labels sheet
    labels = prep_data(pd.read_csv(filename), image_dir)
    
    
    transform = None
    test_dataset = tenX_dataset(labels, image_dir, transform)
    
    #len() check
    length = len(test_dataset)
    expect = 5
    assert length is expect, f'10x dataset length method failed. Got {length}, should be {expect}'
    
    #get_item() check
    samples = []
    keys = ['image', 'plastic','shape','color']
    for i in range(len(test_dataset)):
        assert test_dataset[i]['image'] is not None, 'Got NoneType instead of image'
        isP = test_dataset[i]['plastic']
        assert math.isclose(0,isP) or math.isclose(1,isP), f'plastic not 0 or 1, instead is {isP}'
        length = len(test_dataset[i]['shape'])
        assert length is 4, f'length of shape array not 4, instead is {length}'
        samples.append(test_dataset[i])
        
    shape = samples[0]['shape']
    assert math.isclose(shape[2], 1),  f'wrong shape first image, is {shape}'
    color = samples[0]['color']
    assert math.isclose(color[0], 1), f'wrong color first image, is {color}'
    assert math.isclose(samples[4]['plastic'], 1), 'wrong plastic id 3rd image'
    
test_dataset_class()

In [ ]:
#0 - BASE LINE 82% naive and 88% model test set accuracy
#1 - 9 output CNN1, 18 output CNN2 87% test vs 77% naive
#2 - 3rd linear layer telescoped (12*39*39 -> 2048 -> 256- >2). 93% test, 87% naive
#3 - added a convulutional layer, 12 -> 24 no good
#2d dropout after convolutions led to predicting all pastics, i also read dropouts after convultions is bad, going to try batch normalizations though
#Another paper I read said that non-adaptive optimizers were better for CNN's if thehyperparameters are tunned
#tuned correctly but are much slower so I don't want to even try given it takes me 20min to train already
#adding batchnormalization after convultions improved training results alot, but not test which is weird since
#its supposed to reduce overfitting. Im going to run again overnight